#Instalar as bibliotecas

In [ ]:
!pip install google-cloud-bigquery

In [ ]:
!pip install joblib

#Tratamento da base

In [ ]:
import google.auth
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import joblib


In [ ]:

# Autenticar o usuário
auth.authenticate_user()

# Inicialize um cliente do BigQuery
clientbq = bigquery.Client(project='aulafiap-403000')


In [ ]:
# Consulta SQL para selecionar os dados da tabela
query = """
    SELECT *
    FROM emf-teacher.trabalho_loans.loan_default
"""

query_job = clientbq.query(query)
dados = query_job.to_dataframe()


In [ ]:
dados.columns

Index(['ID', 'year', 'loan_limit', 'Gender', 'approv_in_adv', 'loan_type',
       'loan_purpose', 'Credit_Worthiness', 'open_credit',
       'business_or_commercial', 'loan_amount', 'rate_of_interest',
       'Interest_rate_spread', 'Upfront_charges', 'term', 'Neg_ammortization',
       'interest_only', 'lump_sum_payment', 'property_value',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'income', 'credit_type', 'Credit_Score', 'co_applicant_credit_type',
       'age', 'submission_of_application', 'LTV', 'Region', 'Security_Type',
       'Status', 'dtir1'],
      dtype='object')

In [ ]:
dados.head(
)

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co_applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
1,24932,2019,cf,Male,nopre,type3,p4,l1,nopc,nob/c,...,EQUI,765,EXP,45-54,to_inst,NaN,North,direct,1,NaN
2,25039,2019,cf,Male,nopre,type2,p3,l1,nopc,b/c,...,EQUI,880,EXP,35-44,to_inst,NaN,North,direct,1,NaN
3,25101,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EQUI,880,EXP,45-54,to_inst,NaN,central,direct,1,NaN
4,25110,2019,cf,Male,nopre,type3,p3,l1,nopc,nob/c,...,EQUI,768,EXP,45-54,not_inst,NaN,North,direct,1,NaN


In [ ]:
#Exibir as informações basicas do dataset
dados.describe()

,ID,year,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,income,Credit_Score,LTV,Status,dtir1
count,148670.0,148670.0,148670.0,112231.000000,112031.000000,109028.000000,148629.000000,1.335720e+05,139520.000000,148670.0,133572.000000,148670.0,124549.000000
mean,99224.5,2019.0,331117.743997,4.045476,0.441656,3224.996127,335.136582,4.978935e+05,6957.338876,699.789103,72.746457,0.246445,37.732932
std,42917.476598,0.0,183909.310127,0.561391,0.513043,3251.121510,58.409084,3.599353e+05,6496.586382,115.875857,39.967603,0.430942,10.545435
min,24890.0,2019.0,16500.0,0.000000,-3.638000,0.000000,96.000000,8.000000e+03,0.000000,500.0,0.967478,0.0,5.000000
25%,62057.25,2019.0,196500.0,3.625000,0.076000,581.490000,360.000000,2.680000e+05,3720.000000,599.0,60.474860,0.0,31.000000
50%,99224.5,2019.0,296500.0,3.990000,0.390400,2596.450000,360.000000,4.180000e+05,5760.000000,699.0,75.135870,0.0,39.000000
75%,136391.75,2019.0,436500.0,4.375000,0.775400,4812.500000,360.000000,6.280000e+05,8520.000000,800.0,86.184211,0.0,45.000000
max,173559.0,2019.0,3576500.0,8.000000,3.357000,60000.000000,360.000000,1.650800e+07,578580.000000,900.0,7831.250000,1.0,61.000000


In [ ]:
#Exibir a quantidade de nulos existentes
dados.isnull().sum()

ID                               0
year                             0
loan_limit                    3344
Gender                           0
approv_in_adv                  908
loan_type                        0
loan_purpose                   134
Credit_Worthiness                0
open_credit                      0
business_or_commercial           0
loan_amount                      0
rate_of_interest             36439
Interest_rate_spread         36639
Upfront_charges              39642
term                            41
Neg_ammortization              121
interest_only                    0
lump_sum_payment                 0
property_value               15098
construction_type                0
occupancy_type                   0
Secured_by                       0
total_units                      0
income                        9150
credit_type                      0
Credit_Score                     0
co_applicant_credit_type         0
age                            200
submission_of_applic

In [ ]:
# dados em porcentagem de nulos
((dados.isnull().sum()/dados.shape[0]) * 100)

ID                            0.000000
year                          0.000000
loan_limit                    2.249277
Gender                        0.000000
approv_in_adv                 0.610749
loan_type                     0.000000
loan_purpose                  0.090133
Credit_Worthiness             0.000000
open_credit                   0.000000
business_or_commercial        0.000000
loan_amount                   0.000000
rate_of_interest             24.509989
Interest_rate_spread         24.644515
Upfront_charges              26.664425
term                          0.027578
Neg_ammortization             0.081388
interest_only                 0.000000
lump_sum_payment              0.000000
property_value               10.155378
construction_type             0.000000
occupancy_type                0.000000
Secured_by                    0.000000
total_units                   0.000000
income                        6.154571
credit_type                   0.000000
Credit_Score             

In [ ]:
dados_filtrados = dados.copy()[['open_credit','Credit_Worthiness','loan_purpose','Gender','loan_type','loan_limit','credit_type']]

In [ ]:
((dados_filtrados.isnull().sum()/dados_filtrados.shape[0]) * 100)

open_credit          0.000000
Credit_Worthiness    0.000000
loan_purpose         0.090133
Gender               0.000000
loan_type            0.000000
loan_limit           2.249277
credit_type          0.000000
dtype: float64

In [ ]:
dados_filtrados[['loan_purpose','loan_limit']]

,loan_purpose,loan_limit
0,p1,cf
1,p4,cf
2,p3,cf
3,p4,cf
4,p3,cf
...,...,...
148665,p4,cf
148666,p3,cf
148667,p3,cf
148668,p4,cf


In [ ]:
dados_filtrados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   open_credit        148670 non-null  object
 1   Credit_Worthiness  148670 non-null  object
 2   loan_purpose       148536 non-null  object
 3   Gender             148670 non-null  object
 4   loan_type          148670 non-null  object
 5   loan_limit         145326 non-null  object
 6   credit_type        148670 non-null  object
dtypes: object(7)
memory usage: 7.9+ MB


In [ ]:
dados_filtrados['loan_purpose'].unique()

array(['p1', 'p4', 'p3', 'p2', None], dtype=object)

In [ ]:
dados_filtrados['loan_limit'].unique()

array(['cf', 'ncf', None], dtype=object)

In [ ]:
#Tratamento dos dados vazios
dados_filtrados.loc[:, 'loan_purpose'] = dados_filtrados.loc[:, 'loan_purpose'].fillna('unknown')

In [ ]:
#Tratamento dos dados vazios
dados_filtrados.loc[:, 'loan_limit'] = dados_filtrados.loc[:, 'loan_limit'].fillna('unknown')

In [ ]:
((dados_filtrados.isnull().sum() / dados_filtrados.shape[0]) * 100)

open_credit          0.0
Credit_Worthiness    0.0
loan_purpose         0.0
Gender               0.0
loan_type            0.0
loan_limit           0.0
credit_type          0.0
dtype: float64

In [ ]:
dados_filtrados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   open_credit        148670 non-null  object
 1   Credit_Worthiness  148670 non-null  object
 2   loan_purpose       148670 non-null  object
 3   Gender             148670 non-null  object
 4   loan_type          148670 non-null  object
 5   loan_limit         148670 non-null  object
 6   credit_type        148670 non-null  object
dtypes: object(7)
memory usage: 7.9+ MB


#Escolha do Target

In [ ]:
dados_filtrados['credit_type'].unique(

)

###TARGET TIPO DE CREDITO


array(['EQUI', 'CRIF', 'CIB', 'EXP'], dtype=object)

#Feature Engeneering

In [ ]:
objetos = dados_filtrados.select_dtypes(include=['object'])
objetos = objetos.drop('credit_type',axis=1)


In [ ]:
objetos

,open_credit,Credit_Worthiness,loan_purpose,Gender,loan_type,loan_limit
0,nopc,l1,p1,Male,type2,cf
1,nopc,l1,p4,Male,type3,cf
2,nopc,l1,p3,Male,type2,cf
3,nopc,l1,p4,Male,type1,cf
4,nopc,l1,p3,Male,type3,cf
...,...,...,...,...,...,...
148665,nopc,l1,p4,Sex Not Available,type1,cf
148666,nopc,l1,p3,Sex Not Available,type1,cf
148667,nopc,l1,p3,Sex Not Available,type2,cf
148668,nopc,l1,p4,Sex Not Available,type3,cf


In [ ]:
objetos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   open_credit        148670 non-null  object
 1   Credit_Worthiness  148670 non-null  object
 2   loan_purpose       148670 non-null  object
 3   Gender             148670 non-null  object
 4   loan_type          148670 non-null  object
 5   loan_limit         148670 non-null  object
dtypes: object(6)
memory usage: 6.8+ MB


In [ ]:
le = LabelEncoder()
# Inicialize um DataFrame vazio para armazenar os objetos transformados
objetos_transformados = pd.DataFrame()

for coluna in objetos.columns:
    # Aplique o LabelEncoder em cada coluna e adicione ao DataFrame objetos_transformados
    objetos_transformados[coluna] = le.fit_transform(objetos[coluna])



#Criação do set de feautures (não otimizado) e Target

In [ ]:
X= objetos_transformados

converter_target = {'EQUI': 1 , 'CRIF': 2, 'CIB': 3, 'EXP': 4}
y= dados_filtrados['credit_type'].map(converter_target)


#Criação do Modelo de Arvore Classificatoria

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
modelo = DecisionTreeClassifier()
modelo.fit(X_train, y_train)


DecisionTreeClassifier()

#Salvando o modelo

In [ ]:
joblib.dump(modelo, 'classificacao_modelo.pkl')

['classificacao_modelo.pkl']